In [45]:
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms,models
import matplotlib.pyplot as plt
import numpy as np
import os

In [46]:
class SandelDataset(Dataset):
    def __init__(self,root_dir,transforms=None):
        self.images = []
        self.labels =[]
        self.transforms = transforms
        for label, folder in enumerate(['left','right']):
            folder_path = os.path.join(root_dir,folder)
            for fileName in os.listdir(folder_path):
                img_path = os.path.join(folder_path,fileName)
                self.images.append(img_path)
                self.labels.append(label)
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        img_path = self.images[index]
        label = self.labels[index]
        image = Image.open(img_path).convert("RGB")
        if self.transforms:
            image = self.transforms(image)
        return image, label

In [57]:
curr_dir = os.getcwd()
dirs=curr_dir.split('/')
dirs[-1]='sandel_dataset'

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

test_data_dir = '/'.join(dirs) + '/test/'
train_data_dir = '/'.join(dirs) + '/train/'

train_dataset = SandelDataset(root_dir=train_data_dir,transforms=transform)
test_dataset = SandelDataset(root_dir=test_data_dir,transforms=transform)

# len(train_dataset), len(test_dataset)
train_dataloader = DataLoader(train_dataset,batch_size=32,shuffle=True)
test_dataloader = DataLoader(test_dataset,batch_size=32,shuffle=False)
len(train_dataset), len(test_dataset)
# plt.figure(figsize=(5,5))

# PyTorch stores image tensors in channel-first format (C, H, W)
# but matplotlib expects channel-last format (H, W, C)
# plt.imshow(train_dataloader.dataset.__getitem__(5)[0].permute(1,2,0))


(736, 238)

In [48]:
class VGG16Model(nn.Module):
  def __init__(self):
    # print("VGG16 Model Initialized")
    super(VGG16Model,self).__init__()
    self.backebone = models.vgg16(pretrained =True).features
    self.avgPool = nn.AdaptiveAvgPool2d((7,7))
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(512*7*7,4096),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(4096,2048),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(2048,1024),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(1024,2)
    )
  def forward(self,x):
    x = self.backebone(x)
    x = self.avgPool(x)
    x = self.classifier(x)
    return x


In [53]:
model = VGG16Model()
optimizer = optim.Adam(model.parameters(),lr=0.0001)
loss_fn= nn.CrossEntropyLoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = loss_fn(outputs,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(train_dataloader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

KeyboardInterrupt: 

In [ ]:
model.eval()
val_loss = 0.0
val_correct = 0
val_total = 0

with torch.no_grad():
    for images, labels in test_dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        val_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        val_correct += (preds == labels).sum().item()
        val_total += labels.size(0)

val_loss /= len(test_dataloader.dataset)
val_acc = 100 * val_correct / val_total
print(f"Validation Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")

In [ ]:
images, labels = next(iter(test_dataloader))
images, labels = images.to(device), labels.to(device)
outputs = model(images)
_, preds = torch.max(outputs, 1)
for i in range(4):
    plt.subplot(2,2,i+1)
    plt.imshow(images[i].cpu().permute(1,2,0))
    plt.title(f"Predicted: {'Left' if preds[i]==0 else 'Right'}, Actual: {'Left' if labels[i]==0 else 'Right'}")
    plt.axis('off')
plt.tight_layout()
plt.show()

32